<a href="https://colab.research.google.com/github/RozenCross12/GPT/blob/main/AnimateDiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![visitor][visitor-badge]][visitor-stats]
# **AnimateDiff**
An Unofficial Colab Notebook For [AnimateDiff](https://github.com/guoyww/AnimateDiff/)

Updated 18/07/2023
- Fix `display image count`
- Add function to download multiple model by separating urls with comma

Updated 18/07/2023 (Thanks to [hoalarious](https://github.com/hoalarious)):

*   Dropdown model and LoRa selection
*   Automatically display last # results

[visitor-badge]: https://api.visitorbadge.io/api/visitors?path=Linaqruf-AnimateDiff&label=Visitors&labelColor=%2334495E&countColor=%231ABC9C&style=flat&labelStyle=none
[visitor-stats]: https://visitorbadge.io/status?path=Linaqruf-AnimateDiff


<br>
<table>
    <tr>
    <td><img src="https://github.com/guoyww/AnimateDiff/raw/main/__assets__/animations/model_02/01.gif"></td>
    <td><img src="https://github.com/guoyww/AnimateDiff/raw/main/__assets__/animations/model_02/02.gif"></td>
    <td><img src="https://github.com/guoyww/AnimateDiff/raw/main/__assets__/animations/model_02/03.gif"></td>
    <td><img src="https://github.com/guoyww/AnimateDiff/raw/main/__assets__/animations/model_02/04.gif"></td>
    </tr>
</table>

TO DO:
- [ ] Create my own fork
- [ ] Add new args for custom output path
- [ ] Fix output naming
- [ ] Add new args to save output to `webm` or `mp4`
- [ ] Save new output to `temp_folder` as well for easier `display(img)`
- [ ] Delete pipeline after each module used to clean memory
- [ ] Support another scheduler (?) ~~DDIM bad~~
- [ ] Support external VAE

UPDATE
- Init Image from [talesofai](https://github.com/talesofai/AnimateDiff/)'s fork is not actually working
- Colab free user can use this notebook but only limited to `512x512` as far i know, higher than that got OOM
- for `fp16` support use [dajes](https://github.com/dajes/AnimateDiff/tree/longer_videos)'s fork

In [2]:
# @title ## **1. Install Environment**

import os
import time
import subprocess
import torch
import sys
from IPython.utils import capture
from IPython.display import Image
import fileinput

python_version      = ".".join(sys.version.split(".")[:2])
colablib_path       = f"/usr/local/lib/python{python_version}/dist-packages/colablib"
if not os.path.exists(colablib_path):
    subprocess.run(['pip', 'install', 'git+https://github.com/Linaqruf/colablib'])

import colablib.utils.py_utils as py_utils
from colablib.utils.py_utils import get_filename
from colablib.utils.ubuntu_utils import ubuntu_deps
from colablib.colored_print import cprint, print_line
from colablib.utils.git_utils import validate_repo, clone_repo
from colablib.utils.config_utils import change_line
from colablib.sd_models.downloader import aria2_download

%store -r

root_dir      = "/content"
deps_dir      = os.path.join(root_dir, "deps")
repo_dir      = os.path.join(root_dir, "AnimateDiff")
models_dir    = os.path.join(root_dir, "models")
lora_dir      = os.path.join(root_dir, "LoRA")
diffusers_dir = os.path.join(repo_dir, "models", "StableDiffusion")
motion_dir    = os.path.join(repo_dir, "models", "Motion_Module")
config_dir    = os.path.join(repo_dir, "configs")
script_dir    = os.path.join(repo_dir, "scripts")
samples_dir   = os.path.join(repo_dir, "samples")


repo_dict = {
    "talesofai/AnimateDiff (forked repo, can do initImage)" : "https://github.com/talesofai/AnimateDiff",
    "dajes/AnimateDiff (fork repo, longer video duration)"  : "https://github.com/dajes/AnimateDiff",
    "guoyww/AnimateDiff (original repo, latest update)"     : "https://github.com/guoyww/AnimateDiff",
}

REPO              = "talesofai/AnimateDiff (forked repo, can do initImage)" #@param ["talesofai/AnimateDiff (forked repo, can do initImage)", "dajes/AnimateDiff (fork repo, longer video duration)", "guoyww/AnimateDiff (original repo, latest update)"] {allow-input: true}
BRANCH            = "main"  # @param {type: "string"}
if REPO == "dajes/AnimateDiff (fork repo, longer video duration)":
    BRANCH = "longer_videos"
DIFFUSERS_MODEL   = "runwayml/stable-diffusion-v1-5" # @param {type: "string"}
repo_url          = repo_dict[REPO]

if not BRANCH:
    BRANCH = "main"

with capture.capture_output() as cap:
    for store in ["root_dir", "deps_dir", "repo_dir",
                  "models_dir", "lora_dir", "diffusers_dir", "motion_dir",]:
        %store {store}
    del cap

def install_dependencies():
    gpu_info          = py_utils.get_gpu_info()
    torch_info        = py_utils.get_torch_version()
    ubuntu_command    = ["apt", "install", "-y", "aria2"]
    requirements_cmd  = ['pip', 'install', "omegaconf", "einops", "omegaconf", "safetensors", "diffusers[torch]==0.11.1", "transformers", "moviepy"]

    desired_xformers  = "0.0.20" if '2.0.1+cu118' in torch_info else "0.0.19"
    desired_torch     = "2.0.0"
    torch_command     = ["pip", "install", "torch==2.0.0+cu118", "torchvision==0.15.1+cu118", "torchaudio==2.0.1+cu118", "torchtext==0.15.1", "torchdata==0.6.0", "--extra-index-url", "https://download.pytorch.org/whl/cu118", "-U"]
    xformers_command  = ['pip', 'install', f'xformers=={desired_xformers}', 'triton==2.0.0', "-U"]

    cprint("Installing ubuntu dependencies...", color="green")
    subprocess.run(ubuntu_command, check=True)

    cprint(f"Installing requirements...", color="green")
    subprocess.run(requirements_cmd, check=True, cwd=repo_dir)

    if '2.0.1+cu118' in torch.__version__:
        cprint(f"Installing xformers", desired_xformers, color="green")
        if 'T4' in gpu_info:
            xformers_command = ['pip', 'install', 'https://github.com/Linaqruf/colab-xformers/releases/download/0.0.20/xformers-0.0.20+1d635e1.d20230519-cp310-cp310-linux_x86_64.whl']
            subprocess.run(xformers_command)
        else:
            subprocess.run(xformers_command)
    else:
        cprint(f"Downgrading torch to", desired_torch, color="green")
        subprocess.run(torch_command)
        cprint(f"Installing xformers", desired_xformers, color="green")
        subprocess.run(xformers_command)

def download_model():
    from huggingface_hub.file_download import hf_hub_url

    v14_module = "https://huggingface.co/camenduru/AnimateDiff/resolve/main/mm_sd_v14.ckpt"
    v15_module = "https://huggingface.co/camenduru/AnimateDiff/resolve/main/mm_sd_v15.ckpt"

    file_list = [
        'scheduler/scheduler_config.json',
        'text_encoder/config.json',
        'text_encoder/pytorch_model.bin',
        'tokenizer/merges.txt',
        'tokenizer/special_tokens_map.json',
        'tokenizer/tokenizer_config.json',
        'tokenizer/vocab.json',
        'unet/config.json',
        'unet/diffusion_pytorch_model.bin',
        'vae/config.json',
        'vae/diffusion_pytorch_model.bin',
        'model_index.json'
    ]

    cprint("Downloading Diffusers Model...", color="green")
    for file in file_list:
        if "/" in file:
            diffusers_subdir = file.split('/')[0]
            download_dir     = os.path.join(diffusers_dir, DIFFUSERS_MODEL, diffusers_subdir)
        else:
            download_dir     = os.path.join(diffusers_dir, DIFFUSERS_MODEL)
        file_url = hf_hub_url(repo_id = DIFFUSERS_MODEL, filename=file)
        aria2_download(download_dir=download_dir, filename=get_filename(file_url), url=file_url, quiet=True)

    cprint("Downloading Motion Module...", color="green")
    for module_url in [v14_module, v15_module]:
        aria2_download(download_dir=motion_dir, filename=get_filename(module_url), url=module_url, quiet=True)

def prepare_environment():
    cprint(f"Preparing environment...", color="green")

    os.environ["TF_CPP_MIN_LOG_LEVEL"]    = "3"
    os.environ["SAFETENSORS_FAST_GPU"]    = "1"
    os.environ["PYTHONWARNINGS"]          = "ignore"

def display_results(count):
    samples = os.listdir(samples_dir)
    samples.sort(reverse=True)
    samples = samples[:count]

    for folder in samples:
        path = os.path.join(samples_dir, folder)
        gif_path = os.path.join(path, 'sample.gif')

        if os.path.exists(gif_path):
            img = Image(filename=gif_path)
            display(img)

def state_dict_patch(file_name):
    old_line = "text_model.load_state_dict(text_model_dict)"
    new_line = "text_model.load_state_dict(text_model_dict, strict=False)"
    with fileinput.FileInput(file_name, inplace=True) as file:
        for line in file:
            if old_line in line:
                line = line.replace(old_line, new_line)
            print(line, end='')  # write back to the file

def main():
    os.chdir(root_dir)
    start_time = time.time()

    gpu_info    = py_utils.get_gpu_info(get_gpu_name=True)
    python_info = py_utils.get_python_version()
    torch_info  = py_utils.get_torch_version()

    print_line(80, color="green")
    cprint(f" [-] Current GPU:", gpu_info, color="flat_yellow")
    cprint(f" [-] Python", python_info, color="flat_yellow")
    cprint(f" [-] Torch", torch_info, color="flat_yellow")
    print_line(80, color="green")

    for dir in [models_dir, lora_dir]:
        os.makedirs(dir, exist_ok=True)

    if not os.path.exists(repo_dir):
        cprint(f"Installing AnimateDiff...", color="green")
        clone_repo(repo_url, directory=repo_dir, branch=BRANCH)
    repo_name, current_commit_hash, current_branch = validate_repo(repo_dir)

    cprint(f"Using '{repo_name}' repository...", color="green")
    cprint(f"Branch: {current_branch}, Commit hash: {current_commit_hash}", color="green")

    print_line(80, color="green")

    install_dependencies()
    download_model()
    prepare_environment()

    script_file = os.path.join(repo_dir, "animatediff", "utils", "convert_from_ckpt.py")
    state_dict_patch(script_file)
    elapsed_time = py_utils.calculate_elapsed_time(start_time)

    print_line(80, color="green")
    cprint(f"Finished installation. Took {elapsed_time}.", color="flat_yellow")
    cprint(f"All is done! Go to the next step.", color="flat_yellow")
    print_line(80, color="green")

main()

!pip install huggingface_hub==0.20.3
!pip install huggingface_hub==0.20.3

KeyboardInterrupt: 

In [2]:
!pip install huggingface_hub==0.20.3

In [3]:
# @title ## **2. Download Model**
import os
import warnings
import time
import colablib.utils.py_utils as py_utils
from colablib.colored_print import cprint, print_line
from colablib.sd_models.downloader import download, get_filepath
from colablib.utils.py_utils import get_filename
from IPython import get_ipython

# --- 1. 定義模型與 LoRA 的儲存路徑 ---
# 我們在這裡明確指定路徑，避免受到其他儲存格的影響
root_dir = "/content/"
models_dir = os.path.join(root_dir, "models", "Stable-diffusion")
lora_dir = os.path.join(root_dir, "models", "Lora")

# 確保這些資料夾一定存在
os.makedirs(models_dir, exist_ok=True)
os.makedirs(lora_dir, exist_ok=True)

os.chdir(root_dir)

# --- 2. 使用者介面設定 ---
# @markdown ### **Download Stable Diffusion Model**
# @markdown Use comma separation for multiple URLs, e.g. `url1, url2, url3`.
MODEL_URL   = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors" #@param ["PASTE MODEL URL OR GDRIVE PATH HERE",  "Anything V3.1", "AnyLoRA Default", "AnyLoRA Anime Mix", "ChilloutMix Ni", "Stable Diffusion V1.5"] {allow-input: true}
# @markdown ### **Download LoRA**
# @markdown Specify `LORA_URL` to download LoRA. Leave it empty if you don't want to use it.
LORA_URL    = ""  #@param ["PASTE LORA URL OR GDRIVE PATH HERE"] {allow-input: true}

# --- 3. 下載邏輯 ---
def main():
    warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

    available_models = {
        "Anything V3.1"           : "https://huggingface.co/cag/anything-v3-1/resolve/main/anything-v3-1.safetensors",
        "AnyLoRA Default"         : "https://huggingface.co/Lykon/AnyLoRA/resolve/main/AnyLoRA_bakedVae_fp16_NOTpruned.safetensors",
        "AnyLoRA Anime Mix"       : "https://huggingface.co/Lykon/AnyLoRA/resolve/main/AAM_Anylora_AnimeMix.safetensors",
        "ChilloutMix Ni"          : "https://huggingface.co/naonovn/chilloutmix_NiPrunedFp32Fix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors",
        "Stable Diffusion V1.5"   : "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors",
    }

    REAL_MODEL_URL = available_models.get(MODEL_URL, MODEL_URL)

    MODEL_URLS = REAL_MODEL_URL.split(',') if REAL_MODEL_URL and "PASTE" not in REAL_MODEL_URL else []
    LORA_URLS = LORA_URL.split(',') if LORA_URL and "PASTE" not in LORA_URL else []

    download_targets = { "model": (MODEL_URLS, models_dir), "lora": (LORA_URLS, lora_dir) }

    model_path = ""
    lora_path = ""

    print_line(80, color="green")
    cprint(" [-] Download process initiated...", color="flat_yellow")
    print_line(80, color="green")

    for target, (urls, dst) in download_targets.items():
        for url in urls:
            if url:
                download(url=url.strip(), dst=dst)
                filepath = get_filepath(url, dst)
                if os.path.exists(filepath):
                    if target == "model": model_path = filepath
                    elif target == "lora": lora_path = filepath
                    file_size = py_utils.get_file_size(filepath)
                    cprint(f" [-] Downloaded {target.capitalize()}: {filepath} ({file_size})", color="flat_yellow")
                else:
                    cprint(f" [!] Failed to find downloaded file for URL: {url}", color="flat_red")

    print_line(80, color="green")
    cprint(f"Download process has been successfully executed!", color="flat_yellow")
    print_line(80, color="green")

    # 將結果回傳，而不是在函式內儲存
    return model_path, lora_path

# --- 4. 執行主程式並儲存結果 ---
# 執行 main() 並接收它回傳的路徑
model_path, lora_path = main()

# 在函式外部，使用 %store 儲存變數
ipython = get_ipython()
if model_path:
    ipython.run_line_magic('store', 'model_path')
    cprint(f"✅ Model path stored: {model_path}", color="flat_green")
if lora_path:
    ipython.run_line_magic('store', 'lora_path')
    cprint(f"✅ LoRA path stored: {lora_path}", color="flat_green")

 [-] Download process initiated...
Filename obtained: 'v1-5-pruned-emaonly.safetensors'
Starting download of 'v1-5-pruned-emaonly.safetensors' with aria2c...
Download of 'v1-5-pruned-emaonly.safetensors' completed. Took 0 sec.
 [-] Downloaded Model: /content/models/Stable-diffusion/v1-5-pruned-emaonly.safetensors (3.97 GB)
Download process has been successfully executed!
Stored 'model_path' (str)
✅ Model path stored: /content/models/Stable-diffusion/v1-5-pruned-emaonly.safetensors


!wget -O /content/v1-5-pruned-emaonly.safetensors https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors


In [4]:
!wget -O /content/v1-5-pruned-emaonly.safetensors https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors


--2025-06-16 15:10:27--  https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors
Resolving huggingface.co (huggingface.co)... 108.138.246.67, 108.138.246.71, 108.138.246.79, ...
Connecting to huggingface.co (huggingface.co)|108.138.246.67|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors [following]
--2025-06-16 15:10:27--  https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/66/6f/666f465fa70158515404e8de2c6bc6fe2f90c46f9296293aa14daededeb32c52/6ce0161689b3853acaa03779ec93eafe75a02f4ced659bee03f50797806fa2fa?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27v1-5-pruned-emaonly.safetensors%3B+filename%3D%22v

In [5]:
import os
from pprint import pprint # 引入 pprint 讓輸出更好看

# --- 1. 所有設定已根據你的截圖和下載紀錄填寫完畢 ---

# (已填寫) 根據你的下載紀錄，這就是模型存放的資料夾路徑
models_dir = "/content/"

# (已填寫) 根據你的下載紀錄，這就是模型的完整檔案名稱
selected_model = "v1-5-pruned-emaonly.safetensors"

# (來自你之前的截圖) 專案/系列名稱
project_name = "JinnLo"

# (來自你之前的截圖) 動畫模組
motion_module = "SD15"

# (來自你之前的截圖) LoRA 設定
lora_weight = 1
lora_alpha = 1

# (來自你之前的截圖) 正向提示詞 (prompt)
prompt = "masterpiece, best quality, 1girl, solo, looking at viewer, hands on hips, (short bob:1.2), (purple hair with blue tips:1.3), blue gradient hair, (purple hair:1.2)"

# (來自你之前的截圖) 反向提示詞 (negative_prompt)
negative_prompt = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality"


# --- 2. 程式碼會自動處理路徑 (這部分不需修改) ---

print("正在設定路徑...")
if "selected_model" in globals() and selected_model:
    model_path = os.path.join(models_dir, selected_model)
    print(f"✅ 基礎模型路徑設定成功: {model_path}")
else:
    model_path = ""
    print("❌ 警告：找不到 'selected_model' 變數或該變數為空。")


# --- 3. 整合所有設定到一個字典中 ---

generation_config = {
    "project_name": project_name,
    "model_config": {
        "base_model_path": model_path,
        "motion_module": motion_module,
    },
    "lora_config": {
        "lora_weight": lora_weight,
        "lora_alpha": lora_alpha,
    },
    "prompt_config": {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
    },
    "steps": 25,
    "guidance_scale": 7.5,
}

print("\n" + "="*40)
print("🎉 所有設定已準備就緒！最終設定如下：")
print("="*40)
pprint(generation_config)

正在設定路徑...
✅ 基礎模型路徑設定成功: /content/v1-5-pruned-emaonly.safetensors

🎉 所有設定已準備就緒！最終設定如下：
{'guidance_scale': 7.5,
 'lora_config': {'lora_alpha': 1, 'lora_weight': 1},
 'model_config': {'base_model_path': '/content/v1-5-pruned-emaonly.safetensors',
                  'motion_module': 'SD15'},
 'project_name': 'JinnLo',
 'prompt_config': {'negative_prompt': 'lowres, bad anatomy, bad hands, text, '
                                      'error, missing fingers, extra digit, '
                                      'fewer digits, cropped, worst quality, '
                                      'low quality, normal quality',
                   'prompt': 'masterpiece, best quality, 1girl, solo, looking '
                             'at viewer, hands on hips, (short bob:1.2), '
                             '(purple hair with blue tips:1.3), blue gradient '
                             'hair, (purple hair:1.2)'},
 'steps': 25}


In [6]:
# @title ## **4. Run to update list. Then select LoRA**
import ipywidgets as widgets

lora_list = os.listdir(lora_dir)
lora_list.append("")
selected_lora = widgets.Dropdown(options=lora_list)

selected_lora

Dropdown(options=('',), value='')

In [8]:
# @title ## **5. Inference**
import yaml
import os
from colablib.utils.git_utils import validate_repo
from colablib.colored_print import cprint, print_line
from IPython.display import display, HTML # 引入 display 和 HTML

# @markdown Specify `project_name` for `config.yaml` header
project_name     = "JinnLo" # @param {type: "string"}
# @markdown ### **Model config**
# 這一段的 .value 處理是正確的，我們保持原樣
if "selected_model" in globals():
    model_path       = os.path.join(models_dir, selected_model)
lora_path        = ""
if "selected_lora" in globals() and selected_lora:
    lora_path        = os.path.join(lora_dir, selected_lora.value)
motion_module    = "SD15" # @param ["ALL","SD14", "SD15"]
#@markdown ### **LoRA Config**
lora_weight      = 1 # @param {type: "number"}
lora_alpha       = 1 # @param {type: "number"}
# @markdown ### **Prompt Config**
prompt           = "masterpiece, best quality, 1girl, solo, looking at viewer, hands on hips, (short bob:1.2), (purple hair with blue tips:1.3), blue gradient hair, (purple eyes:1.2), (serious expression:1.1), slight frown, (cropped jacket:1.2), (purple and blue jacket:1.2), black crop top, black pleated skirt, black choker, black belt, (futuristic hair ornaments:1.1), hair clips, (subtle breast bouncing:1.3), (gentle chest movement:1.2), (slight jiggle:1.1), dynamic pose, motion, animation," # @param {type: "string"}
negative_prompt  = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, static, stiff, no movement, unnatural motion," # @param {type: "string"}
image_path       = ""  # @param {type: "string"}
# @markdown ### **Generation Parameter**
steps            = 28  # @param {type: "number"}
guidance_scale   = 12 # @param {type: "number"}
seed             = -1 # @param {type: "number"}
resolution       = "512,512" # @param {type: "string"}
# @markdown ### **Video Duration**
video_length     = 16 # @param {type: "number"}
context_length   = 16 # @param {type: "number"}
context_overlap  = -1 # @param {type: "number"}
context_stride   = 0 # @param {type: "number"}
# @markdown ### **Preview Settings**
show_results     = True #@param {type:"boolean"}
How_many_previous_results_to_show = 1 #@param {type:"number"}

# --- 這部分的程式碼是準備設定檔，我們保持原樣 ---
pretrained_model_path = os.path.join(diffusers_dir, DIFFUSERS_MODEL)
inference_config_file = os.path.join(repo_dir, "configs", "inference", "inference.yaml")
config_file           = os.path.join(config_dir, project_name + ".yaml")

separators = ["*", "x", ","]
for separator in separators:
    if separator in resolution:
        width, height = [value.strip() for value in resolution.split(separator)]
        break

animate_config = { "pretrained_model_path": pretrained_model_path, "inference_config": inference_config_file, "config": config_file, "L": video_length, "W": width, "H": height, }
image_config = { project_name : { "base": "", "path": model_path, "additional_networks": [], "motion_module": [], "steps": steps, "guidance_scale": guidance_scale, "prompt": [], "n_prompt": [] } }
if lora_path.endswith(".safetensors"):
    image_config[project_name]["additional_networks"].append(f"{lora_path} : {lora_weight}")
    image_config[project_name]["lora_alpha"] = lora_alpha
if motion_module == "SD14": image_config[project_name]["motion_module"].append(f"{motion_dir}/mm_sd_v14.ckpt")
elif motion_module == "SD15": image_config[project_name]["motion_module"].append(f"{motion_dir}/mm_sd_v15.ckpt")
else:
    image_config[project_name]["motion_module"].append(f"{motion_dir}/mm_sd_v14.ckpt")
    image_config[project_name]["motion_module"].append(f"{motion_dir}/mm_sd_v15.ckpt")
if image_path: image_config[project_name]["init_image"] = image_path
if seed > 0 : image_config[project_name]["seed"] = seed
if prompt: image_config[project_name]["prompt"].append(prompt)
if negative_prompt: image_config[project_name]["n_prompt"].append(negative_prompt)
if context_length: animate_config["context_length"] = context_length
if context_overlap: animate_config["context_overlap"] = context_overlap
if context_stride: animate_config["context_stride"] = context_stride

def repo_check(repo_dir, animate_config, image_config):
    repo_name, _, _ = validate_repo(repo_dir)
    # ... (repo_check 函式內容保持不變) ...
    return animate_config, image_config

def parse_args(config):
    args = ""
    # ... (parse_args 函式內容保持不變) ...
    return args.strip()

def write_to_yaml(data, filename):
    with open(filename, 'w') as file:
        yaml.dump(data, file)

# --- 這部分的程式碼是執行生成，我們保持原樣 ---
print_line(80, color="green")
cprint(" [-] Initiating AnimateDiff...", color="flat_yellow")
print_line(80, color="green")
if "selected_model" in globals(): cprint(f' Model: {selected_model}', color="flat_yellow")
if "selected_lora" in globals() and selected_lora: cprint(f' LoRA: {selected_lora.value}', color="flat_yellow")
print_line(80, color="green")
animate_config, image_config = repo_check(repo_dir, animate_config, image_config)
animate_args = parse_args(animate_config)
write_to_yaml(image_config, config_file)
final_args = f"python -m scripts.animate {animate_args}"
os.chdir(repo_dir)
!{final_args}

# --- 關鍵修改點 ---
# 在顯示結果之前，明確定義成果資料夾的路徑
# 這樣 display_results 函式才知道要去哪裡找檔案
if show_results:
    cprint(f"[*] Searching for results in project folder '{project_name}'...", color="flat_blue")
    samples_dir = f"samples/{project_name}"

    # 這是顯示結果的函式，我們把它也放在這裡，讓儲存格可以獨立運作
    def display_results(output_dir, count=1):
        if os.path.exists(output_dir):
            files = [f for f in os.listdir(output_dir) if f.endswith(('.gif', '.mp4', '.png'))]
            files.sort(key=lambda f: os.path.getmtime(os.path.join(output_dir, f)), reverse=True)

            if files:
                cprint(f"✅ Found {len(files)} result(s). Displaying the latest {count}.", color="flat_green")
                display_count = min(len(files), count)
                for i in range(display_count):
                    file_path = os.path.join(output_dir, files[i])
                    display(HTML(f'<h4>{files[i]}</h4><video src="{file_path}" controls autoplay loop style="width:100%; max-width:512px;"></video>'))
            else:
                cprint(f"⚠️ No results found in the directory: {output_dir}", color="flat_yellow")
        else:
            cprint(f"❌ Error: The output directory does not exist: {output_dir}", color="flat_red")

    display_results(samples_dir, How_many_previous_results_to_show)

 [-] Initiating AnimateDiff...
 Model: v1-5-pruned-emaonly.safetensors
 LoRA: 
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/AnimateDiff/scripts/animate.py", line 9, in <module>
    import diffusers
  File "/usr/local/lib/python3.11/dist-packages/diffusers/__init__.py", line 27, in <module>
    from .modeling_utils import ModelMixin
  File "/usr/local/lib/python3.11/dist-packages/diffusers/modeling_utils.py", line 52, in <module>
    import accelerate
  File "/usr/local/lib/python3.11/dist-packages/accelerate/__init__.py", line 16, in <module>
    from .accelerator import Accelerator
  File "/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py", line 34, in <module>
    from huggingface_hub import split_torch_state_dict_into_shards
ImportError: cannot import name 'split_torch_state_dict_into_shards' from 'huggingface_hub' (/usr/local/lib/python3.11/dist-packages/h

In [ ]:
# @title ## **6. Display your result**
How_many_previous_results_to_show = 4 #@param {type:"integer"}

display_results(How_many_previous_results_to_show)

In [ ]:
# @title ## **(Optional) Convert to MP4**
from moviepy.editor import *
from IPython.display import HTML
from base64 import b64encode

gif_path = "/content/AnimateDiff/samples/Anime-2023-07-16T03-40-19/sample/0-(Overhead-view),dynamic-angle,ultra-detailed,-illustration,-close-up,-straight-on,-1girl,-(fantasy:1.4),-((purple.gif" #@param {type:'string'}
mp4_output = "/content/output.mp4" #@param {type:'string'}

gif = VideoFileClip(gif_path)
gif.write_videofile(mp4_output, codec='libx264')

mp4 = open(mp4_output,'rb').read()

data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# @title ## **(Optional) Download Generated Animation**
# @markdown Download file manually from files tab or save to Google Drive
import os

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials
from colablib.colored_print import cprint, print_line

%store -r

os.chdir(os.path.join(repo_dir, "samples"))

use_drive = True  # @param {type:"boolean"}
folder_name = "AnimateDiff"  # @param {type: "string"}
filename = "waifu.zip"  # @param {type: "string"}
save_as = filename

name_file = os.path.splitext(filename)[0]
if os.path.exists(filename):
    i = 1
    while os.path.exists(f"{name_file}({i}).zip"):
        i += 1
    filename = f"{name_file}({i}).zip"

os.system('zip -r /content/outputs.zip .')

if use_drive:
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    def create_folder(folder_name):
        file_list = drive.ListFile({
            "q": f"title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
        }).GetList()
        if file_list:
            cprint("Debug: Folder exists", color="green")
            folder_id = file_list[0]["id"]
        else:
            cprint("Debug: Creating folder", color="green")
            file = drive.CreateFile({
                "title": folder_name,
                "mimeType": "application/vnd.google-apps.folder"
            })
            file.Upload()
            folder_id = file.attr["metadata"]["id"]
        return folder_id

    def upload_file(file_name, folder_id, save_as):
        file_list = drive.ListFile({"q": f"title='{save_as}' and trashed=false"}).GetList()
        if file_list:
            cprint("Debug: File already exists", color="green")
            i = 1
            while True:
                new_name = f"{os.path.splitext(save_as)[0]}({i}){os.path.splitext(save_as)[1]}"
                file_list = drive.ListFile({"q": f"title='{new_name}' and trashed=false"}).GetList()
                if not file_list:
                    save_as = new_name
                    break
                i += 1
        file = drive.CreateFile({"title": save_as, "parents": [{"id": folder_id}]})
        file.SetContentFile(file_name)
        file.Upload()
        file.InsertPermission({"type": "anyone", "value": "anyone", "role": "reader"})
        return file.attr["metadata"]["id"]

    file_id = upload_file("/content/outputs.zip", create_folder(folder_name), save_as)
    cprint(f"Your sharing link: https://drive.google.com/file/d/{file_id}/view?usp=sharing", color="green")
